# Tools (도구)

OpenAI 플랫폼에서 사용할 수 있는 도구에 대한 개요는 다음과 같습니다. 
- 함수 호출
- 웹검색
- 원격 MCP 서버
- 파일 검색
- 이미지 생성
- 코드 인터프리터
- 컴퓨터 사용

In [69]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

True

In [70]:
from openai import OpenAI

client = OpenAI()

Model = "gpt-4.1-nano"

### 웹검색 (Web Search)
모델이 응답을 생성하기 전에 최신 정보를 웹에서 검색할 수 있도록 허용

In [73]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-mini",
    tools=[{"type": "web_search_preview"}],
    input="오늘 한국의 종합주가지수는 얼마입니까?"
)

print(response.output_text)

현재 시각은 2025년 6월 30일 월요일 오전 5시 4분입니다. 한국 주식시장은 오전 9시에 개장하므로, 오늘의 종합주가지수는 아직 확인할 수 없습니다. 주식시장은 평일 오전 9시부터 오후 3시 30분까지 운영되며, 한국거래소(KRX)나 주요 금융 포털 사이트에서 실시간 지수를 확인하실 수 있습니다. 


- 사용자 위치 지정

In [75]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="o4-mini",
    tools=[{
        "type": "web_search_preview",
        "user_location": {
            "type": "approximate",
            "country": "KR",
            "city": "서울"
        }
    }],
    input="가로수 길에서 제일 핫한 레스토랑이 어디인가요?",
)

print(response.output_text)

가로수길에서 현재 가장 ‘핫’한 레스토랑으로 꼽히는 곳은 신사동 골목 안에 위치한 이탈리안 다이닝 에스토(ESTO)입니다.

- 위치 및 오픈 시기  
  서울 강남구 압구정로10길 인근 조용한 골목에 자리잡고 있으며, 2025년 초 오픈해 빠르게 입소문을 탔습니다. ([kr.trip.com](https://kr.trip.com/moments/detail/seoul-234-131682063/?utm_source=chatgpt.com))  

- 분위기 & 인테리어  
  유럽 감성이 물씬 느껴지는 세련된 외관과 은은한 조명이 특징으로, 모던하면서도 아늑한 공간 연출이 인상적입니다. ([kr.trip.com](https://kr.trip.com/moments/detail/seoul-234-131682063/?utm_source=chatgpt.com))  

- 메뉴 특징  
  파스타‧리조또‧스테이크 등 클래식 이탈리안 요리를 중심으로, 재료 본연의 맛을 살린 담백하고 깊은 풍미가 인기. 특히 크림 파스타와 쫀득한 리조또, 내추럴 와인 페어링이 호평을 받고 있습니다. ([kr.trip.com](https://kr.trip.com/moments/detail/seoul-234-131682063/?utm_source=chatgpt.com))  

- 인기 포인트  
  • 2025년 5월 1일 트립닷컴 인플루언서 추천 맛집으로 선정  
  • SNS 해시태그(#에스토 #신사동맛집 등)에서 높은 화제성  
  • 데이트·기념일·소규모 모임 최적화된 고급스러운 분위기 ([kr.trip.com](https://kr.trip.com/moments/detail/seoul-234-131682063/?utm_source=chatgpt.com))  

– Tip –  
 • 디너 타임 코스 이용 시 가장 만족도가 높으며, 사전 예약은 필수입니다.  
 • 소규모 모임을 위한 룸 좌석이 있으니 방문 전 문의해 보세요.


# 파일 검색
모델이 응답을 생성하기 전에 관련 정보를 파일에서 검색할 수 있도록 합니다.
파일 검색은 Responses API에서 제공되는 도구입니다. 모델이 시맨틱 및 키워드 검색을 통해 이전에 업로드된 파일의 지식 기반에서 정보를 검색할 수 있도록 합니다. 벡터 저장소를 생성하고 파일을 업로드하면, 모델이 이러한 지식 기반 또는 벡터 저장소에 접근할 수 있도록 하여 모델의 내재된 지식을 강화할 수 있습니다.

이 도구는 OpenAI에서 관리하는 호스팅 도구이므로, 실행을 처리하기 위해 코드를 직접 구현할 필요가 없습니다. 모델이 이 도구를 사용하기로 결정하면 자동으로 도구를 호출하여 파일에서 정보를 검색하고 출력을 반환합니다.

In [76]:
# "Financial Statements"라는 벡터 스토어 생성
vector_store = client.vector_stores.create(name="Financial Statements")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["재무제표/goog-10k.pdf", "재무제표/brka-10k.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [77]:
response = client.responses.create(
    model=Model,
    input="2023년 10월 말에 AAPL의 발행 주식 수는 얼마였나요?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)
print(response)

Response(id='resp_68621bee30f48190b7d30c0108b262d0027ec34bb1c87800', created_at=1751260142.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-nano-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68621bee78b48190bae556ca8e6f3acd027ec34bb1c87800', queries=['2023년 10월 말 AAPL의 발행 주식 수'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68621bef71788190b666abae9eeb100b027ec34bb1c87800', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-2dhYcwq2P4DFuLF6BTovCn', filename='goog-10k.pdf', index=240, type='file_citation')], text='2023년 10월 말 기준으로 AAPL의 발행 주식 수에 대한 구체적인 정보는 제공된 자료에서 확인할 수 없습니다. 다만, 2021년 2월 16일 기준으로 AAPL의 Class A와 Class B, 그리고 Class C 주식의 발행 주식 수는 모두 수백만 주 수준임을 알 수 있으며, 이 시점의 발행 주식 수는 각각 640,586주, 1,336,348,609주(클래스 A와 B의 합계), 그리고 344,756주(클래스 C)였습니다. 2023년 10월 말 기준에 정확한 수치는 별도로 확인이 필요합니다.', type='output_text', logprobs=[])], role='assistant', status=

In [78]:
response.output_text

'2023년 10월 말 기준으로 AAPL의 발행 주식 수에 대한 구체적인 정보는 제공된 자료에서 확인할 수 없습니다. 다만, 2021년 2월 16일 기준으로 AAPL의 Class A와 Class B, 그리고 Class C 주식의 발행 주식 수는 모두 수백만 주 수준임을 알 수 있으며, 이 시점의 발행 주식 수는 각각 640,586주, 1,336,348,609주(클래스 A와 B의 합계), 그리고 344,756주(클래스 C)였습니다. 2023년 10월 말 기준에 정확한 수치는 별도로 확인이 필요합니다.'

In [79]:
response = client.responses.create(
    model=Model,
    input="2023년 Google의 당기 순이익은 얼마였나요? BERKSHIRE HATHAWAY INC. 와 어느 쪽의 당기 순이익이 더 많았나요?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

In [80]:
print(response.output_text)

2023년 Google의 순이익에 대한 직접적인 수치는 제공된 자료에서 찾을 수 없었습니다. 그러나 Berkshire Hathaway Inc.의 2023년 순이익은 약 42,521백만 달러(42.5억 달러)로 보고되었습니다.

반면, Google의 2023년 순이익은 자료에서 명확히 언급되어 있지 않기 때문에 직접적인 비교는 어렵습니다. 하지만, Google의 2022년 또는 2023년 관련자료가 필요하거나 다른 재무자료를 참고하셔야 할 것 같습니다.

요약하자면, Berkshire Hathaway의 2023년 당기 순이익은 약 42.5억 달러이며, Google의 2023년 당기 순이익은 별도로 제공된 자료 내에서 확인되지 않습니다. 따라서 두 회사의 2023년 당기 순이익을 명확히 비교하기는 어렵습니다.


In [81]:
# step2) 새 벡터 스토어 생성
vector_store = client.vector_stores.create(name="기업분석보고서")
 
# OpenAI에 업로드할 파일 준비
file_paths = ["재무제표/네이버.pdf", "재무제표/LG엔솔.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# 업로드 및 폴링 SDK 도우미를 사용하여 파일을 업로드하고 벡터 스토어에 추가,
# 파일 배치의 완료 상태를 폴링합니다.
file_batch = client.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# 이 작업의 결과를 보기 위해 상태 및 파일 개수를 출력할 수 있습니다.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [82]:
response = client.responses.create(
    model=Model,
    input="LG에너지솔루션의 2024년 하반기 전망을 요약해 주세요. 에코프로비엠은 매출액이 줄은 것에 비해 왜 영업이익이 더 크게 줄어들었나요?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print(response.output_text)

LG에너지솔루션의 2024년 하반기 전망은 GM의 신차 판매 성장세가 핵심 변수로 부각됩니다. GM의 판매 증가는 배터리 수요 회복에 중요한 역할을 할 것으로 기대되며, 특히 GM의 Ultium 플랫폼 기반 전기차 판매가 지속적으로 증가하고 있기 때문입니다. 그러나 전통 완성차 업체들의 판매 부진이 심화되고 있어, LG전기의 전반적인 수익성은 하락할 우려도 존재합니다. 투자의견은 유지되고 있으며, 신차 출시와 판매 확대 기대가 주가 회복의 동력으로 작용하는 전망입니다.

에코프로비엠의 경우, 매출액이 줄면서 영업이익이 더 크게 감소한 이유는 다양한 요인이 복합적으로 작용한 결과입니다. 주된 원인으로는 원자재 가격 상승과 공급망 문제로 인한 원가 상승, 경쟁 심화로 인한 가격 압박, 그리고 특정 사업 부문의 판매 부진이 영향을 미쳤을 수 있습니다. 특히, 매출이 줄어들었음에도 불구하고 고정비 부담이 크거나, 생산 설비의 효율성 저하, 또는 신제품 출시에 따른 초기 투자 비용이 영업이익 하락을 가중시킨 것으로 보입니다. 

요약하자면, LG에너지솔루션의 하반기 전망은 GM 신차 판매와 시장 회복세에 달려 있으며, 에코프로비엠은 매출 저하와 함께 원가 상승 및 사업 구조적 문제로 영업이익이 더 크게 감소하는 상황임을 알 수 있습니다.


In [83]:
response = client.responses.create(
    model=Model,
    input="2023년 4분기의 네이버의 영업이익은 얼마였나요? 한화 투자 증권이 목표가를 상향 조정한 이유는 무엇인가요?",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store.id]
    }]
)

print(response.output_text)

2023년 4분기 네이버의 영업이익은 약 4055억 원으로 보고되었습니다. 한화투자증권이 네이버의 목표주가를 상향 조정한 이유는, 광고 및 커머스 수익성 개선세와 웹툰 지분 가치 상향, 그리고 비용 통제에 따른 이익 개선세가 반영되었기 때문입니다. 특히, 광고/커머스 분야의 실적 개선 기대와 웹툰 엔터의 상장 모멘텀 부각이 추가적인 기업 가치 상승의 잠재력으로 작용할 것으로 보입니다.


### 코드 인터프리터 (Code Interpreter)
모델이 Python을 작성하고 실행하여 문제를 해결할 수 있도록 지원합니다.  

코드 인터프리터 도구를 사용하면 모델이 샌드박스 환경에서 Python 코드를 작성하고 실행하여 데이터 분석, 코딩, 수학 등의 복잡한 문제를 해결할 수 있습니다. 다음과 같은 용도로 사용할 수 있습니다.

- 다양한 데이터 및 형식이 포함된 파일 처리
- 데이터와 그래프 이미지가 포함된 파일 생성
- 문제 해결을 위해 반복적으로 코드를 작성하고 실행합니다. 예를 들어, 실행에 실패한 코드를 작성한 모델은 성공할 때까지 해당 코드를 계속 다시 작성하고 실행할 수 있습니다.

최신 추론 모델인 o3와 o4-mini는 코드 인터프리터를 사용하여 이미지를 심층적으로 이해하도록 훈련되었습니다. 자르기, 확대, 회전 및 기타 이미지 처리 기술을 수행하여 시각적 지능을 향상시킬 수 있습니다.

In [84]:
resp = client.responses.create(
  model=Model,
  tools=[
    {
      "type": "code_interpreter",
      "container": { "type": "auto" }
    }
  ],
  instructions="당신은 개인 수학 과외 선생님입니다. 수학 문제가 나오면 코드를 작성하고 실행하여 질문에 답하세요.",
  input="3x + 11 = 14 방정식을 풀어야 합니다. 도와주실 수 있나요?",
)

print(resp.output_text)

물론이죠! 방정식을 풀기 위해서는 \( x \)의 값을 구하면 됩니다. 먼저, 방정식을 정리해 볼게요:

\[ 3x + 11 = 14 \]

양쪽에서 11을 빼서 \( x \)에 대해 풀어보겠습니다. 이를 파이썬 코드로 해결해 보겠습니다.방정식 \( 3x + 11 = 14 \)의 해는 \( x = 1 \)입니다.


In [85]:
resp

Response(id='resp_68621c000c84819588f08150b3e3eaf00a1b52c23d1b68ca', created_at=1751260160.0, error=None, incomplete_details=None, instructions='당신은 개인 수학 과외 선생님입니다. 수학 문제가 나오면 코드를 작성하고 실행하여 질문에 답하세요.', metadata={}, model='gpt-4.1-nano-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_68621c0197dc8195a30668dce7537b0a0a1b52c23d1b68ca', content=[ResponseOutputText(annotations=[], text='물론이죠! 방정식을 풀기 위해서는 \\( x \\)의 값을 구하면 됩니다. 먼저, 방정식을 정리해 볼게요:\n\n\\[ 3x + 11 = 14 \\]\n\n양쪽에서 11을 빼서 \\( x \\)에 대해 풀어보겠습니다. 이를 파이썬 코드로 해결해 보겠습니다.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message'), ResponseCodeInterpreterToolCall(id='ci_68621c0232ac81959dabae304f08a4c40a1b52c23d1b68ca', code="from sympy import symbols, Eq, solve\n\nx = symbols('x')\nequation = Eq(3*x + 11, 14)\nsolution = solve(equation, x)\nsolution", container_id='cntr_68621c0166788190b3ff0d7fe9fe90260442aeaed19ff974', outputs=None, status='completed', type='code_interpreter_call'

-----------------
`resp.output` 에 List 형식으로 추론 단계가 저장되어 있습니다.

In [86]:
print(resp.output[0].content[0].text)
print("------------------------------------------------------------------------------------------")
print(resp.output[1].code)
print("------------------------------------------------------------------------------------------")
print(resp.output[2].content[0].text)

물론이죠! 방정식을 풀기 위해서는 \( x \)의 값을 구하면 됩니다. 먼저, 방정식을 정리해 볼게요:

\[ 3x + 11 = 14 \]

양쪽에서 11을 빼서 \( x \)에 대해 풀어보겠습니다. 이를 파이썬 코드로 해결해 보겠습니다.
------------------------------------------------------------------------------------------
from sympy import symbols, Eq, solve

x = symbols('x')
equation = Eq(3*x + 11, 14)
solution = solve(equation, x)
solution
------------------------------------------------------------------------------------------
방정식 \( 3x + 11 = 14 \)의 해는 \( x = 1 \)입니다.


## 실습: 코드 인터프리터와 파일 검색 도구 활용

다음과 같은 사용자 질문이 주어졌을 때, OpenAI Responses API를 이용해 코드 인터프리터와 벡터 스토어 기반 검색 도구를 적절히 활용하여 응답을 생성하는 실습을 하세요.

1. **수학 문제 풀이**
   사용자 질문:

> `"x^2 - 5x + 6 = 0 방정식의 해를 구해주세요."`

- `code_interpreter` 도구를 사용하여 문제를 코드로 풀고, 결과를 사용자에게 출력하세요.
- instructions를 사용하여 모델에게 “수학 선생님 역할”을 지정하세요.


2. **PDF 기반 질문 응답 (file\_search 도구)**
   PDF 파일 `삼성전자_사업보고서.pdf`를 벡터 스토어에 업로드하고, 다음 질문에 답하도록 구현하세요:

사용자 질문:

> `"2023년 삼성전자의 반도체 부문 실적 요약을 알려주세요."`

- 벡터 스토어를 생성하고 파일을 업로드하는 코드
- file\_search 도구를 통해 답을 생성하는 응답 흐름 구현
